<a href="https://colab.research.google.com/github/sharlynmuturi/Pytorch-Tutorial/blob/main/CNN_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ============================================
# 1. IMPORTS
# ============================================

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
# CIFAR-10 standard normalization values
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),   # data augmentation
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_data = datasets.CIFAR10(root='cifar', train=True, download=True,
                              transform=transform_train)
test_data = datasets.CIFAR10(root='cifar', train=False, download=True,
                             transform=transform_test)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)


100%|██████████| 170M/170M [00:03<00:00, 45.6MB/s]


In [5]:
# Load ResNet-18
model = models.resnet18()

# Replace the final layer for CIFAR-10 (10 classes)
model.fc = nn.Linear(model.fc.in_features, 10)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)

# Learning rate scheduler (very important!)
scheduler = optim.lr_scheduler.StepLR(optimizer,
                                      step_size=30, gamma=0.1)


In [8]:
def train_epoch():
    model.train()
    total, correct, total_loss = 0, 0, 0

    for images, labels in train_loader:

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return total_loss/len(train_loader), correct/total


def test_epoch():
    model.eval()
    total, correct, total_loss = 0, 0, 0

    with torch.no_grad():
        for images, labels in test_loader:

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return total_loss/len(test_loader), correct/total


In [ ]:
train_losses = []
test_losses = []
train_accs = []
test_accs = []

epochs = 60

for epoch in range(epochs):
    train_loss, train_acc = train_epoch()
    test_loss, test_acc = test_epoch()
    scheduler.step()   # update LR

    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accs.append(train_acc)
    test_accs.append(test_acc)

    print(f"Epoch {epoch+1}/{epochs} "
          f"| Train Acc: {train_acc:.4f} "
          f"| Test Acc: {test_acc:.4f}")


In [ ]:
plt.figure(figsize=(12,5))
plt.plot(train_accs, label='Train Acc')
plt.plot(test_accs, label='Test Acc')
plt.legend()
plt.title("Accuracy per Epoch")
plt.show()

plt.figure(figsize=(12,5))
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.legend()
plt.title("Loss per Epoch")
plt.show()


In [ ]:
print(f"\nFinal Test Accuracy: {test_accs[-1]*100:.2f}%")